In [14]:
import json
import numpy as np
from pathlib import Path
import pandas as pd
import random

def read_jsonl(file_path):
    # List to store the extracted data
    data = []

    # Read and process the JSONL file
    with open(file_path, 'r') as file:
        for line in file:
            json_data = json.loads(line)  # Parse each line as JSON
            custom_id = json_data.get("custom_id")  # Extract the custom_id
            
            # Access the nested "content" field inside "choices"
            choices = json_data.get("response", {}).get("body", {}).get("choices", [])
            for choice in choices:
                message = choice.get("message", {})
                content = message.get("content")  # Extract the text content
                
                if content:
                    # Append the custom_id and content to the data list
                    data.append({"custom_id": custom_id, "text": content})

    # Convert the data into a DataFrame
    df = pd.DataFrame(data)
    return df

#helper lambda function
def extract_vote(text):
    if "[[yes]]" in text and "[[no]]" in text:
        return pd.NA
    elif "[[yes]]" in text:
        return 1
    elif "[[no]]" in text:
        return 0
    else:
        return pd.NA

def extract_data(df):
    #extract the index so we can do a join operation later on 
    df['index'] = df['custom_id'].str.split('-').str[-1].astype(int)

    #extract the batch number for solumn naming 
    num = df['custom_id'].str.split('-').str[-2].astype(int)[0]

    #set index
    df = df.set_index("index").drop(columns=["custom_id"])

    #rename
    df  = df.rename(columns={"text" : f"Text_{num}"})

    #get binary votes from models 
    df[f'Vote_{num}'] = df[f'Text_{num}'].apply(lambda x: extract_vote(x))
    return df


def join_votes(df_list): 
    df = pd.concat(df_list, axis="columns")
    df = df[["Vote_0", "Vote_1", "Vote_2", "Vote_3", "Vote_4", "Vote_5", "Vote_6", "Vote_7", "Vote_8", "Vote_9", "Vote_10"]]
    df["Proportion_Yes"] = df.mean(axis="columns")
    df["Proportion_Yes_10"] = df[["Vote_0", "Vote_1", "Vote_2", "Vote_3", "Vote_4", "Vote_5", "Vote_6", "Vote_7", "Vote_8", "Vote_9"]].mean(axis='columns')
    
    #
    df["Proportion_Yes_9a"] = df[["Vote_0", "Vote_1", "Vote_2", "Vote_3", "Vote_4", "Vote_5", "Vote_6", "Vote_7", "Vote_8"]].mean(axis='columns')
    df["Proportion_Yes_9b"] = df[["Vote_1", "Vote_2", "Vote_3", "Vote_4", "Vote_5", "Vote_6", "Vote_7", "Vote_8", "Vote_9"]].mean(axis='columns')
    
    df["Proportion_Yes"] = df.mean(axis="columns")

    df["Majority_Vote"] = df["Proportion_Yes"].apply(lambda x: 1 if x > 0.5 else 0 if x < 0.5 else pd.NA)

    #Fix cases where this is still a tie!
    df.loc[df["Majority_Vote"].isna(), "Majority_Vote"] = df["Proportion_Yes_10"].apply(lambda x: 1 if x > 0.5 else 0 if x < 0.5 else pd.NA)
    random.seed(537)
    if random.random() > 0.5: 
        df.loc[df["Majority_Vote"].isna(), "Majority_Vote"] = df["Proportion_Yes_9a"].apply(lambda x: 1 if x > 0.5 else 0 if x < 0.5 else pd.NA)
    else: 
        df.loc[df["Majority_Vote"].isna(), "Majority_Vote"] = df["Proportion_Yes_9b"].apply(lambda x: 1 if x > 0.5 else 0 if x < 0.5 else pd.NA)


    df = df.drop(columns=['Proportion_Yes_10', 'Proportion_Yes_9a', 'Proportion_Yes_9b'])

    return df

# Path to the folder
folder_path = Path('../results/gpt/')

# Get all file paths in the folder
file_paths = list(folder_path.rglob('*')) 


df_list = []
for path in file_paths:
    df = read_jsonl(path)
    df = extract_data(df)
    df_list.append(df)
df = join_votes(df_list)

df

,Vote_0,Vote_1,Vote_2,Vote_3,Vote_4,Vote_5,Vote_6,Vote_7,Vote_8,Vote_9,Vote_10,Proportion_Yes,Majority_Vote
index,,,,,,,,,,,,,
0,1,1,1,1,1,1,1,1,1,1,1,1.000000,1
1,0,0,0,0,1,0,1,1,0,0,1,0.355354,0
2,1,1,1,1,1,1,1,1,1,1,1,1.000000,1
3,1,1,1,1,1,1,1,1,1,1,1,1.000000,1
4,1,1,1,1,1,1,1,1,1,1,1,1.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2492,0,0,1,1,0,0,1,1,1,1,1,0.630572,1
2493,0,0,0,1,1,1,0,0,1,0,1,0.449562,0
2494,1,1,0,1,0,1,1,1,1,0,1,0.724781,1


In [15]:
#Finding errors
votes = df[["Vote_0", "Vote_1", "Vote_2", "Vote_3", "Vote_4", "Vote_5", "Vote_6", "Vote_7", "Vote_8", "Vote_9", "Vote_10"]]
print(votes.isna().sum())

Vote_0     6
Vote_1     1
Vote_2     1
Vote_3     2
Vote_4     1
Vote_5     4
Vote_6     2
Vote_7     3
Vote_8     1
Vote_9     2
Vote_10    3
dtype: int64


In [16]:
df.mean()

Vote_0            0.724207
Vote_1            0.727564
Vote_2            0.727965
Vote_3            0.731062
Vote_4            0.725561
Vote_5            0.745688
Vote_6            0.752705
Vote_7            0.744988
Vote_8            0.748798
Vote_9            0.753507
Vote_10           0.745790
Proportion_Yes    0.738620
Majority_Vote     0.748498
dtype: float64

In [17]:
ballotq = pd.read_csv('../data/clean_ballot_qs.csv', index_col=0)
result = pd.concat([ballotq, df], axis='columns')
result = result[["Year", "State", "Topic_Areas", "Election", "Percent_Yes", "Pass", "Proportion_Yes", "Majority_Vote"]]
result

,Year,State,Topic_Areas,Election,Percent_Yes,Pass,Proportion_Yes,Majority_Vote
0,2000,Alabama,Civil & Constitutional Law,General,0.600,1,1.000000,1
1,2000,Alabama,Judiciary,General,0.570,1,0.355354,0
2,2000,Alabama,Local Government,General,0.630,1,1.000000,1
3,2000,Alabama,Human Services,General,0.570,1,1.000000,1
4,2000,Alabama,Bond Measures,General,0.630,1,1.000000,1
...,...,...,...,...,...,...,...,...
2492,2014,Wyoming,Education: Higher Ed,General,0.297,0,0.630572,1
2493,2016,Wyoming,Budgets,General,0.563,1,0.449562,0
2494,2020,Wyoming,Energy & Electric Utilities,General,0.510,1,0.724781,1
2495,2022,Wyoming,Judiciary,General,0.390,0,0.811582,1


In [18]:
np.sum(result["Majority_Vote"].isna())

0

In [8]:
no_ties = result.dropna(subset="Majority_Vote")
print("Among results without ties, percentage of LLM responses aligning with voters")
print(np.mean(no_ties["Pass"]==no_ties["Majority_Vote"]))

TypeError: Index(...) must be called with a collection of some kind, 'Majority_Vote' was passed